In [ ]:
import sys
from PyQt5 import QtWidgets
from PyQt5 import QtGui
from PyQt5 import uic
from PyQt5 import QtCore
from PyQt5.QtCore import pyqtSlot
from PyQt5.QtCore import QThread
import urllib.request
from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#import time

class GoogleThread(QThread):
    def __init__(self, n, search, result, directory_path, parent=None): 
        QThread.__init__(self)
        self.n = n
        self.search = search
        self.result = result
        self.directory_path = directory_path

    # Google
    def run(self):
        driver = './chromedriver/chromedriver.exe'
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument("disable-gpu")
        browser = webdriver.Chrome(driver, chrome_options=options)
    
        browser.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&ei=l1AdWbegOcra8QXvtr-4Cw&ved=0EKouCBUoAQ")
        elem = browser.find_element_by_xpath("//*[@class='gLFyf gsfi']") 
        elem.send_keys(self.search.toPlainText())
        elem.submit()
        
        for i in range(10):
            browser.find_element_by_xpath("//body").send_keys(Keys.END)
            self.sleep(5)
            try: browser.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
            except: pass
            self.result.setText(f'{i+1} 페이지를 조회중입니다.\n조회 완료까지 {9-i}페이지 남았습니다.')

        html = browser.page_source
        soup = BeautifulSoup(html, "lxml")

        def fetch_list_url():
            params = []
            imgList = soup.find_all("img", class_="rg_i")
            for im in imgList:
                try :
                    params.append(im["src"])
                except KeyError:
                    params.append(im["data-src"])
            return params

        self.sleep(2)
        
        def fetch_detail_url():
            params = fetch_list_url()
            for idx, p in enumerate(params,1):
                urllib.request.urlretrieve(p, self.directory_path.toPlainText() + '/' + str(idx) + ".jpg")
                self.result.setText(f'{idx}장의 사진이 저장되었습니다.')
            self.result.setText(self.result.text() + '\n작업이 완료되었습니다.')
            
        fetch_detail_url()
        browser.quit()  

#     # Naver
#     def download_naver(self):
#         pass
    
#     # Bing
#     def download_bing(self):
#         pass

class Image_Scrapper(QtWidgets.QDialog):
    def __init__(self, parent=None):
        QtWidgets.QDialog.__init__(self, parent)
        self.ui = uic.loadUi('Image_Scrapper.ui', self)
        self.ui.show()
        # 어플리케이션 이름
        self.setWindowTitle('Image Scrapper')
        # 어플리케이션 아이콘
        self.setWindowIcon(QtGui.QIcon('app_icon.jpg'))
        # github 이미지
        #self.github.setStyleSheet('image:url(github.png);border:0px;')
        # 경로 plain text
        self.directory_path.clear()
        # 결과 progress bar
        self.progress.setValue(10)

        # 폴더 선택 버튼 클릭
        self.select_folder.clicked.connect(self.select_folder_clicked)
        # 사진 저장 버튼 클릭
        self.download.clicked.connect(self.download_clicked)
        # engine radio 선택
        self.google.clicked.connect(self.radio_clicked)
        self.naver.clicked.connect(self.radio_clicked)
        self.bing.clicked.connect(self.radio_clicked)
        # copyright box 잠금
        self.copyright_box.setEnabled(False)
        # copyright radio 선택
        self.google_0.clicked.connect(self.copyright_clicked)
        self.google_1.clicked.connect(self.copyright_clicked)
        self.google_2.clicked.connect(self.copyright_clicked)
        self.google_3.clicked.connect(self.copyright_clicked)
        self.google_4.clicked.connect(self.copyright_clicked)
        
        
    # 폴더 선택 버튼 클릭 시 동작
    def select_folder_clicked(self):
        fname = QtWidgets.QFileDialog.getExistingDirectory()
        self.directory_path.setPlainText(fname)
    
    # 사진 저장 버튼 클릭 시 동작
    def download_clicked(self):
        if not self.search.toPlainText(): QtWidgets.QMessageBox.about(self, '경고', "검색어를 입력하세요.")
        elif not self.directory_path.toPlainText(): QtWidgets.QMessageBox.about(self, '경고', "저장 경로를 선택하세요.")
        else:
            select_engine = [i.isChecked() for i in self.engine_box.children()]
            if not select_engine.count(True): QtWidgets.QMessageBox.about(self, '경고', "검색 엔진을 선택하세요.")
            elif select_engine.index(True) == 0: 
                select_copyright = [i.isChecked() for i in self.copyright_box.children()]
                if not select_copyright.count(True): QtWidgets.QMessageBox.about(self, '경고', "Google 이미지 사용권을 선택하세요.")
                #else: self.download_google(select_copyright.index(True))
                else:
                    # google thread
                    self.googlethread = GoogleThread(select_copyright.index(True), self.search, self.result, self.directory_path)
                    self.googlethread.start()
            elif select_engine.index(True) == 1: self.download_naver(self)
            elif select_engine.index(True) == 2: self.download_bing(self)

    # engine radio 선택 시 동작
    def radio_clicked(self, engine):
        if self.google.isChecked():
            self.copyright_box.setEnabled(True)
        else:
            reply = QtWidgets.QMessageBox.question(self, '경고', '저작권 문제가 있을 수 있습니다.\n계속 진행하시겠습니까?',
                                                   QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No)
            if reply == QtWidgets.QMessageBox.No:
                self.google.setChecked(True)
                self.copyright_box.setEnabled(True)
            else:
                self.copyright_box.setEnabled(False)
            
    # copyright radio 선택 시 동작
    def copyright_clicked(self, n):
        if self.google_0.isChecked():
            reply = QtWidgets.QMessageBox.question(self, '경고', '저작권 문제가 있을 수 있습니다.\n계속 진행하시겠습니까?',
                                                   QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No)
            if reply == QtWidgets.QMessageBox.No:
                self.google_1.setChecked(True)
    
    



#     # github 버튼 클릭 시 동작
#     def github_clicked(self):
#         import webbrowser
#         webbrowser.open('https://github.com/IllIIIllll/calculate')
        
        
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    w = Image_Scrapper()
    app.exec_()

C:\Users\jejsm\anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options


AttributeError: 'QLabel' object has no attribute 'append'

In [2]:
######################################################
import urllib.request
from  bs4 import BeautifulSoup
from selenium import webdriver  # 웹 애플리케이션의 테스트를 자동화하기 위한 프레임 워크
from selenium.webdriver.common.keys import Keys
import time                     # 중간중간 sleep 을 걸어야 해서 time 모듈 import
########################### url 받아오기 ###########################

# 웹브라우져로 크롬을 사용할거라서 크롬 드라이버를 다운받아 위의 위치에 둔다
# 팬텀 js로 하면 백그라운드로 실행할 수 있음
# Chrome 옵션 추가
options = webdriver.ChromeOptions()
# headless
options.add_argument('headless')
# disable GPU
options.add_argument("--disable-gpu")
binary = 'c:/chromedriver/chromedriver.exe'

# 브라우져를 인스턴스화
browser = webdriver.Chrome(binary, chrome_options=options)

# 구글의 이미지 검색 url 받아옴(아무것도 안 쳤을때의 url)
browser.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&ei=l1AdWbegOcra8QXvtr-4Cw&ved=0EKouCBUoAQ")

# 구글의 이미지 검색에 해당하는 input 창의 id 가 '  ?  ' 임(검색창에 해당하는 html코드를 찾아서 elem 사용하도록 설정)
# input창 찾는 방법은 원노트에 있음

#elem = browser.find_elements_by_class_name('gLFyf gsfi')

elem = browser.find_element_by_xpath("//*[@class='gLFyf gsfi']") 



########################### 검색어 입력 ###########################

# elem 이 input 창과 연결되어 스스로 햄버거를 검색
elem.send_keys("햄버거")

# 웹에서의 submit 은 엔터의 역할을 함
elem.submit()

########################### 반복할 횟수 ###########################

# 스크롤을 내리려면 브라우져 이미지 검색결과 부분(바디부분)에 마우스 클릭 한번 하고 End키를 눌러야함
for i in range(1, 5):
    browser.find_element_by_xpath("//body").send_keys(Keys.END)
    time.sleep(10)                  # END 키 누르고 내려가는데 시간이 걸려서 sleep 해줌

time.sleep(10)                      # 네트워크 느릴까봐 안정성 위해 sleep 해줌
html = browser.page_source         # 크롬브라우져에서 현재 불러온 소스 가져옴

soup = BeautifulSoup(html, "lxml") # html 코드를 검색할 수 있도록 설정

########################### 그림파일 저장 ###########################

def fetch_list_url():
    params = []
    imgList = soup.find_all("img", class_="rg_i")  # 구글 이미지 url 이 있는 img 태그의 _img 클래스에 가서
    for im in imgList:
        try :
            params.append(im["src"])                   # params 리스트에 image url 을 담음
        except KeyError:
            params.append(im["data-src"])
    return params

# 이미지의 상세 url 의 값이 있는 src 가 없을 경우
# data-src 로 가져오시오 ~ 


def fetch_detail_url():
    params = fetch_list_url()

    for idx,p in enumerate(params,1):
        # 다운받을 폴더경로 입력
        urllib.request.urlretrieve(p, "C:/111/" + str(idx) + ".jpg")

# enumerate 는 리스트의 모든 요소를 인덱스와 쌍으로 추출
# 하는 함수 . 숫자 1은 인덱스를 1부터 시작해라 ~

fetch_detail_url()

# 끝나면 브라우져 닫기
browser.quit()

D:\JiSung\anaconda\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


KeyboardInterrupt: 

In [ ]:
######################################################
import urllib.request
from  bs4 import BeautifulSoup
from selenium import webdriver  # 웹 애플리케이션의 테스트를 자동화하기 위한 프레임 워크
from selenium.webdriver.common.keys import Keys
import time                     # 중간중간 sleep 을 걸어야 해서 time 모듈 import
########################### url 받아오기 ###########################

# 웹브라우져로 크롬을 사용할거라서 크롬 드라이버를 다운받아 위의 위치에 둔다
# 팬텀 js로 하면 백그라운드로 실행할 수 있음
binary = 'c:/chromedriver/chromedriver.exe'

# 브라우져를 인스턴스화
browser = webdriver.Chrome(binary)

# 구글의 이미지 검색 url 받아옴(아무것도 안 쳤을때의 url)
browser.get("https://search.naver.com/search.naver?where=image&amp;sm=stb_nmr&amp;")

# 구글의 이미지 검색에 해당하는 input 창의 id 가 '  ?  ' 임(검색창에 해당하는 html코드를 찾아서 elem 사용하도록 설정)
# input창 찾는 방법은 원노트에 있음

#elem = browser.find_elements_by_class_name('gLFyf gsfi')

elem = browser.find_element_by_xpath('//*[@id="nx_query"]') 



########################### 검색어 입력 ###########################

# elem 이 input 창과 연결되어 스스로 햄버거를 검색
elem.send_keys("햄버거")

# 웹에서의 submit 은 엔터의 역할을 함
elem.submit()

########################### 반복할 횟수 ###########################

# 스크롤을 내리려면 브라우져 이미지 검색결과 부분(바디부분)에 마우스 클릭 한번 하고 End키를 눌러야함
for i in range(1, 5):
    browser.find_element_by_xpath("//body").send_keys(Keys.END)
    time.sleep(10)                  # END 키 누르고 내려가는데 시간이 걸려서 sleep 해줌

time.sleep(10)                      # 네트워크 느릴까봐 안정성 위해 sleep 해줌
html = browser.page_source         # 크롬브라우져에서 현재 불러온 소스 가져옴

soup = BeautifulSoup(html, "lxml") # html 코드를 검색할 수 있도록 설정

########################### 그림파일 저장 ###########################

def fetch_list_url():
    params = []
    imgList = soup.select("a.thumb._thumb > img")  # 구글 이미지 url 이 있는 img 태그의 _img 클래스에 가서
    for im in imgList:
        try :
            params.append(im["src"])                   # params 리스트에 image url 을 담음
        except KeyError:
            params.append(im["data-src"])
    return params

# 이미지의 상세 url 의 값이 있는 src 가 없을 경우
# data-src 로 가져오시오 ~ 


def fetch_detail_url():
    params = fetch_list_url()

    for idx,p in enumerate(params,1):
        # 다운받을 폴더경로 입력
        urllib.request.urlretrieve(p, "C:/naverimages/" + str(idx) + ".jpg")

# enumerate 는 리스트의 모든 요소를 인덱스와 쌍으로 추출
# 하는 함수 . 숫자 1은 인덱스를 1부터 시작해라 ~

fetch_detail_url()

# 끝나면 브라우져 닫기
browser.quit()

In [ ]:
######################################################
import urllib.request
from  bs4 import BeautifulSoup
from selenium import webdriver  # 웹 애플리케이션의 테스트를 자동화하기 위한 프레임 워크
from selenium.webdriver.common.keys import Keys
import time                     # 중간중간 sleep 을 걸어야 해서 time 모듈 import
########################### url 받아오기 ###########################

# 웹브라우져로 크롬을 사용할거라서 크롬 드라이버를 다운받아 위의 위치에 둔다
# 팬텀 js로 하면 백그라운드로 실행할 수 있음
binary = 'c:/chromedriver/chromedriver.exe'

# 브라우져를 인스턴스화
browser = webdriver.Chrome(binary)

# 구글의 이미지 검색 url 받아옴(아무것도 안 쳤을때의 url)
browser.get("https://www.bing.com/images?FORM=Z9LH")

# 구글의 이미지 검색에 해당하는 input 창의 id 가 '  ?  ' 임(검색창에 해당하는 html코드를 찾아서 elem 사용하도록 설정)
# input창 찾는 방법은 원노트에 있음

#elem = browser.find_elements_by_class_name('gLFyf gsfi')

elem = browser.find_element_by_xpath('//*[@id="sb_form_q"]') 



########################### 검색어 입력 ###########################

# elem 이 input 창과 연결되어 스스로 햄버거를 검색
elem.send_keys("햄버거")

# 웹에서의 submit 은 엔터의 역할을 함
elem.submit()

########################### 반복할 횟수 ###########################

# 스크롤을 내리려면 브라우져 이미지 검색결과 부분(바디부분)에 마우스 클릭 한번 하고 End키를 눌러야함
for i in range(1, 5):
    browser.find_element_by_xpath("//body").send_keys(Keys.END)
    time.sleep(10)                  # END 키 누르고 내려가는데 시간이 걸려서 sleep 해줌

time.sleep(10)                      # 네트워크 느릴까봐 안정성 위해 sleep 해줌
html = browser.page_source         # 크롬브라우져에서 현재 불러온 소스 가져옴

soup = BeautifulSoup(html, "lxml") # html 코드를 검색할 수 있도록 설정

########################### 그림파일 저장 ###########################

def fetch_list_url():
    params = []
    imgList = soup.select("div > div > a > div > img")  # 구글 이미지 url 이 있는 img 태그의 _img 클래스에 가서
    for im in imgList:
        try :
            params.append(im["src"])                   # params 리스트에 image url 을 담음
        except KeyError:
            params.append(im["data-src"])
    return params

# 이미지의 상세 url 의 값이 있는 src 가 없을 경우
# data-src 로 가져오시오 ~ 


def fetch_detail_url():
    params = fetch_list_url()

    for idx,p in enumerate(params,1):
        # 다운받을 폴더경로 입력
        urllib.request.urlretrieve(p, "C:/bingimages/" + str(idx) + ".jpg")

# enumerate 는 리스트의 모든 요소를 인덱스와 쌍으로 추출
# 하는 함수 . 숫자 1은 인덱스를 1부터 시작해라 ~

fetch_detail_url()

# 끝나면 브라우져 닫기
browser.quit()